## Feature testing - Model loss 

In [4]:
# libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
# read in data
all_stats_df = pd.DataFrame(pd.read_csv('../resources/transformed_data/all_stats.csv'))
all_stats_df = all_stats_df.drop(columns=['WTeamID'])

# drop all NaNs
all_stats_df = all_stats_df.dropna(how='any')

all_stats_df.head()

,Season,DayNum,Team1,Team2,Tourney,WLoc,ScoreDiff,Team1Seed,Team2Seed,Team1FirstYear,...,AvgStl_Team1,AvgStl_Team2,AvgBlk_Team1,AvgBlk_Team2,AvgPF_Team1,AvgPF_Team2,FG%_Team1,FG%_Team2,FG3%_Team1,FG3%_Team2
74048,2003,10,1104,1328,0,0,6,10,1,1985,...,6.607143,6.933333,3.785714,3.766667,18.035714,18.600000,0.420362,0.446934,0.320144,0.393673
74049,2003,10,1272,1393,0,0,7,7,3,1985,...,7.379310,8.310345,5.068966,7.275862,18.758621,16.586207,0.437931,0.470067,0.348797,0.330435
74050,2003,11,1266,1437,0,0,12,3,30,1985,...,6.000000,7.500000,3.642857,3.400000,18.642857,20.900000,0.483810,0.420429,0.379391,0.349040
74051,2003,11,1296,1457,0,0,6,30,30,1985,...,7.612903,7.607143,3.612903,5.392857,19.806452,19.642857,0.458967,0.432107,0.383104,0.351687
74052,2003,11,1208,1400,0,0,6,30,1,1985,...,7.629630,6.392857,4.370370,3.857143,17.185185,20.357143,0.464135,0.448513,0.380252,0.348936


In [82]:
# starter DF to reset to each iteration
starter_df = all_stats_df[['Season', 
                           'Team1', 'Team2',
                           'WLoc','WTeam',
                           'Tourney',
                           'Team1RankMean','Team2RankMean',
                           'WinCount_Team1', 'WinCount_Team2', 
#                            'Team1Seed', 'Team2Seed', 
#                            'Win%_Team1','Win%_Team2', 
#                            'LoseCount_Team1', 'LoseCount_Team2',
#                            'FG3%_Team1', 'FG3%_Team2',
#                            'AvgFGM_Team1', 'AvgFGM_Team2',
                           'GameCount_Team1', 'GameCount_Team2',
                          ]]
starter_df

,Season,Team1,Team2,WLoc,WTeam,Tourney,Team1RankMean,Team2RankMean,WinCount_Team1,WinCount_Team2,GameCount_Team1,GameCount_Team2
74048,2003,1104,1328,0,1,0,27.655502,15.730233,17.0,24.0,28.0,30.0
74049,2003,1272,1393,0,1,0,42.000000,25.596154,23.0,24.0,29.0,29.0
74050,2003,1266,1437,0,1,0,18.967442,60.385000,23.0,15.0,28.0,30.0
74051,2003,1296,1457,0,1,0,147.512500,209.452500,17.0,18.0,31.0,28.0
74052,2003,1208,1400,0,0,0,19.261283,9.416279,19.0,22.0,27.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...
168791,2014,1163,1277,0,1,1,26.517978,10.352550,26.0,26.0,34.0,34.0
168792,2014,1246,1276,0,1,1,20.905134,22.406109,24.0,25.0,34.0,33.0
168793,2014,1163,1196,0,1,1,26.517978,11.246120,26.0,32.0,34.0,34.0
168794,2014,1246,1458,0,1,1,20.905134,9.019978,24.0,26.0,34.0,33.0


In [83]:
# list of columns to iterate through
other_cols = ['Team1FirstYear', 'Team2FirstYear',
             'Team1Seed', 'Team2Seed',
#              'Team1RankMean', 'Team2RankMean', 
#              'WinCount_Team1', 'WinCount_Team2', 
#              'GameCount_Team1', 'GameCount_Team2', 
             'AvgScore_Team1', 'AvgScore_Team2',                              
             'Win%_Team1','Win%_Team2', 
             'LoseCount_Team1', 'LoseCount_Team2',
             'AvgFGM_Team1',  'AvgFGM_Team2', 
             'AvgFGA_Team1', 'AvgFGA_Team2', 
             'AvgFGM3_Team1', 'AvgFGM3_Team2', 
             'AvgFGA3_Team1', 'AvgFGA3_Team2', 
             'AvgOR_Team1', 'AvgOR_Team2', 
             'AvgDR_Team1', 'AvgDR_Team2', 
             'FG%_Team1', 'FG%_Team2', 
             'FG3%_Team1', 'FG3%_Team2'
             ]

## Build model pipeline

In [57]:
# One hot encoder and scaler for pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

ct = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include=np.number)),
    (OneHotEncoder(handle_unknown='ignore'), make_column_selector(dtype_include=object))
)

In [87]:
# build pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn import metrics

mlc = MLPClassifier(alpha=0.001, hidden_layer_sizes=(168,), 
              random_state=1, solver='sgd',
              nesterovs_momentum=True)

pipe = make_pipeline(ct, mlc)

## Initial fit

In [88]:
# first fit

# select features and target
features_df = starter_df.copy()
target = features_df.pop('WTeam')
selected_features = features_df

X_train, X_test, y_train, y_test = train_test_split(selected_features, 
                                                    target, stratify=target, 
                                                    random_state=42)

pipe.fit(X_train, y_train)

preds = pipe.predict(X_test)
loss_score = metrics.log_loss(y_test, preds)

print('\n-------------------')
print(f'Log loss: {loss_score}')
print('---------------------')
print(metrics.confusion_matrix(y_test,preds))
print('\n-------------------')
print(metrics.classification_report(y_test,preds))
print('-------------------\n')


-------------------
Log loss: 8.484030613383828
---------------------
[[9205 2815]
 [2925 8423]]

-------------------
              precision    recall  f1-score   support

           0       0.76      0.77      0.76     12020
           1       0.75      0.74      0.75     11348

    accuracy                           0.75     23368
   macro avg       0.75      0.75      0.75     23368
weighted avg       0.75      0.75      0.75     23368

-------------------



log loss 1: 8.520980999224033

log loss 2: 8.39830485868785

log loss 3: 8.367263816334052

Current log loss: Log loss: 8.314055601531692

## Team1 and Team2 stat fits

In [11]:
# feature iteration
effective_features = {}

for col in other_cols[::2]:
    iteration_df = starter_df.copy()
    team1_stat = col
    team2_stat = other_cols[other_cols.index(col)+1]
    print(f'{team1_stat} - {team2_stat}')
    iteration_df[team1_stat] = all_stats_df[team1_stat]
    iteration_df[team2_stat] = all_stats_df[team2_stat]

    # select features and target
    target = iteration_df.pop('WTeam')
    selected_features = iteration_df

    X_train, X_test, y_train, y_test = train_test_split(selected_features, 
                                                        target, stratify=target, 
                                                        random_state=42)

    pipe.fit(X_train, y_train)

    preds = pipe.predict(X_test)
    new_loss_score = metrics.log_loss(y_test, preds)

    print(f'Log loss: {new_loss_score}')
    print('---------------------')
    print(metrics.confusion_matrix(y_test,preds))
    print('-------------------')
    print(metrics.classification_report(y_test,preds))
    print('-------------------\n\n')
    
    # add effective features to dict
    if new_loss_score < loss_score:
#         loss_score = new_loss_score
        effective_features[team1_stat] = new_loss_score - loss_score

Team1FirstYear - Team2FirstYear
Log loss: 15.964368658544355
---------------------
[[9465 2555]
 [8246 3102]]
-------------------
              precision    recall  f1-score   support

           0       0.53      0.79      0.64     12020
           1       0.55      0.27      0.36     11348

    accuracy                           0.54     23368
   macro avg       0.54      0.53      0.50     23368
weighted avg       0.54      0.54      0.50     23368

-------------------


Team1Seed - Team2Seed
Log loss: 13.466461374104854
---------------------
[[10172  1848]
 [ 7263  4085]]
-------------------
              precision    recall  f1-score   support

           0       0.58      0.85      0.69     12020
           1       0.69      0.36      0.47     11348

    accuracy                           0.61     23368
   macro avg       0.64      0.60      0.58     23368
weighted avg       0.63      0.61      0.58     23368

-------------------


Team1RankMean - Team2RankMean
Log loss: 8.606709

In [12]:
effective_features

{'Team1FirstYear': -0.9178712094867549,
 'Team1Seed': -3.415778493926256,
 'Team1RankMean': -8.275530787312686,
 'WinCount_Team1': -7.84689701805973,
 'GameCount_Team1': -3.9995315544951318,
 'AvgScore_Team1': -4.512407395203274,
 'Win%_Team1': -7.789250961537725,
 'LoseCount_Team1': -7.449296078930887,
 'AvgFGM_Team1': -4.14732967609358,
 'AvgFGA_Team1': -0.8454046423632988,
 'AvgFGM3_Team1': -1.5430197175340776,
 'AvgFGA3_Team1': -0.3148160322560969,
 'AvgOR_Team1': -0.8113906902728765,
 'AvgDR_Team1': -3.0964399997671492,
 'FG%_Team1': -4.923306701083757,
 'FG3%_Team1': -3.1274766622651136}

In [13]:
# return most effective loss-reducing feature
import operator
min(effective_features.items(), key=operator.itemgetter(1))[0]

'Team1RankMean'

## Stat difference fits

In [84]:
# feature iteration
effective_features = {}

for col in other_cols[::2]:
    iteration_df = starter_df.copy()
    team1_stat = col
    team2_stat = other_cols[other_cols.index(col)+1]
    print(f'{team1_stat}- {team2_stat}')
    iteration_df['TestColumn']=\
        all_stats_df[team1_stat] - all_stats_df[team2_stat]

    # select features and target
    target = iteration_df.pop('WTeam')
    selected_features = iteration_df

    X_train, X_test, y_train, y_test = train_test_split(selected_features, 
                                                        target, stratify=target, 
                                                        random_state=42)

    pipe.fit(X_train, y_train)

    preds = pipe.predict(X_test)
    new_loss_score = metrics.log_loss(y_test, preds)

    print(f'Log loss: {new_loss_score}')
    print('---------------------')
    print(metrics.confusion_matrix(y_test,preds))
    print('-------------------')
    print(metrics.classification_report(y_test,preds))
    print('-------------------\n\n')
    
    # add effective features to dict
    if new_loss_score < loss_score:
        effective_features[team1_stat] = new_loss_score - loss_score
        loss_score = new_loss_score

Team1FirstYear- Team2FirstYear
Log loss: 8.373176170971345
---------------------
[[9253 2767]
 [2898 8450]]
-------------------
              precision    recall  f1-score   support

           0       0.76      0.77      0.77     12020
           1       0.75      0.74      0.75     11348

    accuracy                           0.76     23368
   macro avg       0.76      0.76      0.76     23368
weighted avg       0.76      0.76      0.76     23368

-------------------


Team1Seed- Team2Seed
Log loss: 8.385000606504939
---------------------
[[9249 2771]
 [2902 8446]]
-------------------
              precision    recall  f1-score   support

           0       0.76      0.77      0.77     12020
           1       0.75      0.74      0.75     11348

    accuracy                           0.76     23368
   macro avg       0.76      0.76      0.76     23368
weighted avg       0.76      0.76      0.76     23368

-------------------


AvgScore_Team1- AvgScore_Team2
Log loss: 8.3628292937239

In [85]:
effective_features

{}

In [81]:
# return most effective loss-reducing feature
import operator
min(effective_features.items(), key=operator.itemgetter(1))[0]

'GameCount_Team1'